# SciKit-Learn Pipeline (All IBD)

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    recall_score,
    auc,
    roc_curve,
    roc_auc_score,
)
import shap

from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    GroupKFold,
)
from sklearn.preprocessing import StandardScaler

from src.data.preprocessing import load_and_preprocess_data
from src.config.constants import NUMERICAL_FEATURES, RANDOM_SEED
from src.config.sklearn_models import models_and_params
from src.data.splitting import split_data_for_sklearn
import json

/Users/chershiongchuah/Developer/machine_learning_for_ibd_fatigue/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pipeline Configuration

In [ ]:
biochemical_remission = False
random_seed = RANDOM_SEED

if biochemical_remission:
    output_path = "output/biochem_remission/"
    cmap = "berlin"
    file_prefix = "biochem_remission"
    df = load_and_preprocess_data(biochemical_remission=True)
else:
    df = load_and_preprocess_data(biochemical_remission=False)
    output_path = "output/all_ibd/"
    cmap = "seismic"
    file_prefix = "all_ibd"

plt.rcParams["font.family"] = ["Roboto", "Arial", "sans-serif"]

## Data Loading

In [3]:
numerical_features = NUMERICAL_FEATURES
scaler = StandardScaler()
scaler.fit(df[numerical_features])
df[numerical_features] = scaler.transform(df[numerical_features])

## Hyperparameter Tuning

Find the best hyperparameters for each of the six models

In [4]:
X_train, X_test, y_train, y_test, groups = split_data_for_sklearn(df)

# Models and parameter grids
models_and_params = models_and_params

# GroupKFold cross-validation method with 5 splits
cv = GroupKFold(n_splits=5)

# Results storage
results = {}

for model_name, (model, param_grid) in models_and_params.items():
    print(f"Running GridSearch for {model_name}...")
    grid = GridSearchCV(
        estimator=model, param_grid=param_grid, cv=cv, n_jobs=-1, scoring="roc_auc"
    )
    grid.fit(
        X_train, y_train, groups=groups
    )  # Pass groups to ensure grouping of study_id into either train or validate sets

    # Extract best parameters, mean, and standard deviation of test scores
    results[model_name] = {
        "best_params": grid.best_params_,
        "mean_score": grid.cv_results_["mean_test_score"][grid.best_index_],
        "std_score": grid.cv_results_["std_test_score"][grid.best_index_],
        "grid": grid,  # Save the grid object for future use
    }

# Display Results
results_df = pd.DataFrame(results).T
print(results_df)

Running GridSearch for Random Forest...
Running GridSearch for Logistic Regression...
Running GridSearch for AdaBoost...
Running GridSearch for XGBoost...
Running GridSearch for SVC...
Running GridSearch for MLPClassifier...
                                                           best_params  \
Random Forest        {'max_depth': 5, 'min_samples_split': 2, 'n_es...   
Logistic Regression     {'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}   
AdaBoost                   {'learning_rate': 0.1, 'n_estimators': 100}   
XGBoost              {'learning_rate': 0.01, 'max_depth': 3, 'n_est...   
SVC                                          {'C': 10, 'gamma': 0.001}   
MLPClassifier        {'activation': 'tanh', 'hidden_layer_sizes': (...   

                    mean_score std_score  \
Random Forest         0.767779  0.017777   
Logistic Regression   0.761771  0.021266   
AdaBoost              0.764821  0.009992   
XGBoost               0.762286  0.010251   
SVC                   0.767342  0.02

## Final Model Training and Evaluation

In [ ]:
# Dictionary to store fitted models and their AUC scores, including mean and SD
fitted_models = {}
roc_results = {}

# Loop through the models and evaluate mean and SD of AUC
for model_name, (model, _) in models_and_params.items():
    print(f"Evaluating {model_name}...")

    # Get best parameters from grid search
    best_params = results[model_name]["best_params"]

    # Initialize the model with best parameters
    best_model = model.set_params(**best_params)

    # Perform cross-validation with GroupKFold
    cv_auc_scores = cross_val_score(
        best_model, X_train, y_train, cv=cv, groups=groups, scoring="roc_auc", n_jobs=-1
    )

    # Compute mean and SD of AUC
    mean_auc = np.mean(cv_auc_scores)
    std_auc = np.std(cv_auc_scores)

    # Fit the model to the entire training set
    best_model.fit(X_train, y_train)

    # Store the fitted model
    fitted_models[model_name] = best_model

    # Predict probabilities for the test set
    y_test_proba = best_model.predict_proba(X_test)[:, 1]

    # Compute ROC curve and AUC on the test set
    fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)
    test_auc = auc(fpr, tpr)

    # Store the results for plotting
    roc_results[model_name] = {
        "fpr": fpr,
        "tpr": tpr,
        "test_auc": test_auc,
        "mean_auc": mean_auc,
        "std_auc": std_auc,
    }


In [ ]:
# Plot ROC curves for all models with mean and SD of AUC
plt.figure(figsize=(10,8))
plt.rcParams["font.family"] = "Roboto"
for model_name, roc_data in roc_results.items():
    mean_auc = roc_data["mean_auc"]
    std_auc = roc_data["std_auc"]
    test_auc = roc_data["test_auc"]
    plt.plot(
        roc_data["fpr"],
        roc_data["tpr"],
        label=f"{model_name} (Test AUC = {test_auc:.2f})",
    )

# Add in Keras DNN Model

# Load the model
if biochemical_remission:
    read_dir = "output/dnn_biochem_remission/"
else:
    read_dir = "output/dnn/"

dnn_fpr = np.loadtxt(f"{read_dir}dnn_fpr.txt")
dnn_tpr = np.loadtxt(f"{read_dir}/dnn_tpr.txt")

with open(f"{read_dir}dnn_metrics.json", "r") as f:
    dnn_metrics = json.load(f)

dnn_auc = dnn_metrics["auc"]
        
plt.plot(dnn_fpr, dnn_tpr, label=f"Deep Neural Network (Test AUC = {dnn_auc:.2f})", linewidth=4)

# Add baseline and plot details
plt.plot([0, 1], [0, 1], "k--", label="Chance (AUC = 0.50)")
plt.xlabel("False Positive Rate", fontsize=20, fontweight="bold")
plt.ylabel("True Positive Rate", fontsize=20, fontweight="bold")

if biochemical_remission:
    plt.title("DNN outperforms other ML algorithms (Biochemical Remission)", fontsize=24, fontweight="bold", pad=40)
else:
    plt.title("DNN outperforms other ML algorithms (All IBD)", fontsize=24, fontweight="bold", pad=40)
plt.legend(loc="lower right", fontsize=14)


# Remove the top and right spines
ax = plt.gca()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

ax.xaxis.label.set_fontweight("bold")
ax.yaxis.label.set_fontweight("bold")
ax.tick_params(axis="both", which="major", labelsize=16, width=2)


# Define the absolute path
save_path = f"{output_path}{file_prefix}_roc_curves.png"

# Save the plot to the specified path
plt.savefig(save_path, dpi=600, bbox_inches="tight")

# Display the plot
plt.show()

### Model Metrics Stored in CSV Table

In [ ]:
# Dictionary to store metrics for each model
metrics_results = []

for model_name, best_model in fitted_models.items():
    print(f"Evaluating metrics for {model_name}...")

    # Predict on the test set
    y_test_pred = best_model.predict(X_test)
    y_test_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for AUC

    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_test_pred)
    sensitivity = recall_score(y_test, y_test_pred)  # TPR
    specificity = tn / (tn + fp)  # TNR
    auc_score = roc_auc_score(y_test, y_test_proba)

    # Store results
    metrics_results.append(
        {
            "Model": model_name,
            "Accuracy": accuracy,
            "Sensitivity (Recall)": sensitivity,
            "Specificity": specificity,
            "AUC": auc_score,
        }
    )

# Add in DNN Metrics Here
metrics_results.append(
    {
        "Model": "Deep Neural Network",
        "Accuracy": dnn_metrics["accuracy"],
        "Sensitivity (Recall)": dnn_metrics["sensitivity"],
        "Specificity": dnn_metrics["specificity"],
        "AUC": dnn_metrics["auc"],
    }
)
# Convert to DataFrame
metrics_df = pd.DataFrame(metrics_results).sort_values("AUC", ascending=False)

metrics_df = metrics_df.round(3)

# Save to CSV
output_file = f"{output_path}{file_prefix}_model_metrics.csv"
metrics_df.to_csv(output_file, index=False)

print(f"Metrics saved to {output_file}")

## SHAP Plots for Each Model

### Random Forest

In [ ]:
# Retrieve the GridSearchCV object for Random Forest
rf_grid = results["Random Forest"]["grid"]

# Retrieve the best Random Forest model
rf_best_model = rf_grid.best_estimator_

# SHAP Analysis
explainer = shap.TreeExplainer(rf_best_model)
shap_values = explainer.shap_values(X_test)

# Extract SHAP values
shap_values_class_1 = shap_values[:, :, 1]

# Correct feature names as a list
feature_names = X_test.columns.tolist()

# Plot SHAP summary plot with correct feature names
shap.summary_plot(
    shap_values_class_1, X_test, feature_names=feature_names, show=False, cmap=cmap
)

plt.title("Random Forest", fontsize=20, pad=20, loc="left")

save_path = f"{output_path}shap_random_forest.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")

plt.show()


### Logistic Regression

In [ ]:
# Retrieve the GridSearchCV object for Logistic Regression
logreg_grid = results["Logistic Regression"]["grid"]

# Retrieve the best Logistic Regression model
logreg_best_model = logreg_grid.best_estimator_

# SHAP plot for logistic regression on test set
masker = shap.maskers.Independent(data=X_test)
explainer = shap.LinearExplainer(logreg_best_model, masker=masker)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(
    shap_values, X_test, feature_names=X_test.columns, show=False, cmap=cmap
)

plt.title("Logistic Regression", fontsize=20, pad=20, loc="left")
plt.xlim(-1, 1)

save_path = f"{output_path}shap_logistic_regression.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

### AdaBoost

In [ ]:
# Retrieve the GridSearchCV object for AdaBoost
adaboost_grid = results["AdaBoost"]["grid"]

# Retrieve the best Logistic Regression model
adaboost_best_model = adaboost_grid.best_estimator_

X_test_sample = shap.sample(X_test, 50)
explainer = shap.KernelExplainer(adaboost_best_model.predict_proba, X_test_sample)
shap_values = explainer.shap_values(X_test_sample)


In [ ]:
shap_values_class_1 = shap_values[:, :, 1]
shap.summary_plot(
    shap_values_class_1,
    X_test_sample,
    feature_names=X_test_sample.columns,
    show=False,
    cmap=cmap,
)

plt.title("AdaBoost", fontsize=20, pad=20, loc="left")

save_path = f"{output_path}shap_adaboost.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

### XGBoost

In [ ]:
# Retrieve the GridSearchCV object for AdaBoost
xgboost_grid = results["XGBoost"]["grid"]

# Retrieve the best Logistic Regression model
xgboost_best_model = xgboost_grid.best_estimator_

explainer = shap.Explainer(xgboost_best_model, X_test)
shap_values = explainer(X_test)
shap.plots.beeswarm(shap_values, max_display=20, show=False, color=plt.get_cmap(cmap))
plt.title("XGBoost", fontsize=20, pad=20, loc="left")

save_path = f"{output_path}shap_xgboost.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

### MLPClassifier

In [ ]:
# Retrieve the GridSearchCV object for MLPClassifier
mlp_grid = results["MLPClassifier"]["grid"]

mlp_best_model = mlp_grid.best_estimator_

X_test_sample = shap.sample(X_test, 50)
explainer = shap.KernelExplainer(mlp_best_model.predict_proba, X_test_sample)
shap_values = explainer.shap_values(X_test_sample)


In [ ]:
shap_values_class_1 = shap_values[:, :, 1]
shap.summary_plot(
    shap_values_class_1,
    X_test_sample,
    feature_names=X_test_sample.columns,
    show=False,
    cmap=cmap,
)

plt.title("MLP Classifier", fontsize=20, pad=20, loc="left")

save_path = f"{output_path}shap_mlp.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()

### SVC

In [ ]:
# Retrieve the GridSearchCV object for MLPClassifier
svc_grid = results["SVC"]["grid"]

svc_best_model = svc_grid.best_estimator_

X_test_sample = shap.sample(X_test, 40)
explainer = shap.KernelExplainer(svc_best_model.predict_proba, X_test_sample)
shap_values = explainer.shap_values(X_test_sample)


In [ ]:
shap_values_class_1 = shap_values[:, :, 1]
shap.summary_plot(
    shap_values_class_1,
    X_test_sample,
    feature_names=X_test_sample.columns,
    show=False,
    max_display=20,
    cmap=cmap,
)

plt.title("Support Vector Classifier", fontsize=20, pad=20, loc="left")

save_path = f"{output_path}shap_svc.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
plt.show()